In [1]:
import numpy as np
import matplotlib.pylab as plt
%matplotlib inline
from example_object import Example_object

import ipyparallel as ipp

# Need to start up engines with something like:
# ipcluster start -n 4

In [2]:

rc = ipp.Client()
# view of all the engines, so can do the same thing on all of them at once
dview = rc[:]

n_engines = len(rc)
n_engines

4

In [3]:
# load the example object on all the engines
dview.execute('from example_object import Example_object')
dview.execute('thingy = Example_object()')

<AsyncResult: execute>

In [4]:
dview['thingy']

In [5]:
# Put some data on each engine
np.random.seed(42)
n_points = 100
for engine in rc:
    engine['data'] = np.random.randn(100)

In [6]:
# Read the data into each thingy object
dview.execute('thingy.read_data(data)')

<AsyncResult: execute>

In [7]:
# Let's move over the kwargs we will want to use as well
for i, engine in enumerate(rc):
    engine['scale'] = i

In [8]:
# Now, call the compute method everywhere in parallel!
# Put a blocking on it so we don't try to access the results before they are complete
dview.execute('result = thingy.compute(scale)', block=True)

<AsyncResult: execute:finished>

In [9]:
# Look at the results
dview['result']

[0.0, 0.94888864051064614, 2.1576956924908157, 2.6390094073654891]

In [10]:
# gather info about each object
dview.execute('info = thingy.hw')
dview['info']

['hello world!', 'hello world!', 'hello world!', 'hello world!']